In [338]:
# Import
import os
import re
import pandas as pd
import cclib.io as cc
from rdkit import Chem
from rdkit.Chem import AllChem

The dataframe excel file with all the molecules is converted into a pandas dataframe.

In [339]:
#############################################################
# LOAD INPUT DATA
#############################################################

# define relative path to input dataframe
CX3_dataframe = ".\\dataframes\\CX3_radical_dataframe.xlsx"
# read file into pandas DataFrame object
df = pd.read_excel(CX3_dataframe)
df = df.sort_values(["atomA","atomB","atomC"])

  atomA atomB atomC  nH  nF  nCl  Cx_in  Cy_in  Cz_in  xA_in  ...  sdI  eprC  \
0     H     H     H   3   0    0    NaN    NaN    NaN    NaN  ...  NaN   NaN   
1     F     H     H   2   1    0    NaN    NaN    NaN    NaN  ...  NaN   NaN   
2    Cl     H     H   2   0    1    NaN    NaN    NaN    NaN  ...  NaN   NaN   
3     F     F     H   1   2    0    NaN    NaN    NaN    NaN  ...  NaN   NaN   
4    Cl     F     H   1   1    1    NaN    NaN    NaN    NaN  ...  NaN   NaN   
5    Cl    Cl     H   1   0    2    NaN    NaN    NaN    NaN  ...  NaN   NaN   
6     F     F     F   0   3    0    NaN    NaN    NaN    NaN  ...  NaN   NaN   
7    Cl     F     F   0   2    1    NaN    NaN    NaN    NaN  ...  NaN   NaN   
8    Cl    Cl     F   0   1    2    NaN    NaN    NaN    NaN  ...  NaN   NaN   
9    Cl    Cl    Cl   0   0    3    NaN    NaN    NaN    NaN  ...  NaN   NaN   

                          eprH  eprF  eprCl  eprBr  eprI  \
0                          NaN   NaN    NaN    NaN   NaN   

Next step is to iterate through all of the rows of the dataframe where each row corresponds to a different molecule. First a parsed_rows array is initialized to store the initial coordinates for each molecule. At the end of every loop the coordinates for a molecule are inserted into a pandas series then the series is appended to the parsed_rows array. To start a smiles string is generated using the column labels as keywords {atomA}, {atomB}, and {atomC}. A molecule object is then generated from the smiles string then Hs are explicitly added and embedded into the molecule object. Next a pandas series is created where each each atom is followed by its x, y, and z coordinates. The order of the atoms in the mol_xyz series is atomA, Carbon, AtomB, and AtomC. The CH3 radical is the only exception to that rule where the carbon atom is moved to the first position when adding hydrogens. Therefore, we set atomA, atomB, and atomC to the 0th, 8th, and 12th index then check manually to see if C is in the first position. If C is atomA then we swap Carbon and its coordinates with the second atom. There may be a more general solution that ensures the correct atoms are connected properly. However, the current solution is simpler than saving each molecule to a MOL file and extracting connectivity information from the bonds block.

Then all the values in the series are set to variables and ensure they are the correct data type before loading into a dictionary. The variables are then assigned keys of the same name so they can be simply loaded into the parsed_row pandas series. The parsed_row is appended to to the parsed_rows list defined at the beginning of the block. Once all the rows of the excel dataframe are parsed, the parsed_rows list is converted to a pandas dataframe for easier manipulation.

In [340]:
#############################################################
# GENERATE INPUT XYZ COORDINATES
#############################################################


# loop through experiment dataframe and generate xyz input coordinates
parsed_rows = []
for _, row in df.iterrows():
    # use rdkit to convert SMILES to XYZ
    smiles = "[{atomA}][C]([{atomB}])[{atomC}]".format(**row)
    print(smiles)
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol) # Adds modifications (the hydrogens are added) to the molecule file
    mol_xyz = pd.Series(Chem.MolToXYZBlock(mol)[1:].split())
    # Check to see if atomA and C coordinates were swapped during the AddHs method then swap their positions
    atomA, atomB, atomC = mol_xyz[0], mol_xyz[8], mol_xyz[12]
    if atomA == 'C':
        i = 0
        while i <= 3:
            mol_xyz[i], mol_xyz[i+4] = mol_xyz[i+4], mol_xyz[i]
            i+=1
    else:
        pass

    # extract data from the molecule xyz coordinate file and convert numbers to strings
    atomA, atomB, atomC = mol_xyz[0], mol_xyz[8], mol_xyz[12]
    Cx_in, Cy_in, Cz_in = float(mol_xyz[5]), float(mol_xyz[6]), float(mol_xyz[7])
    xA_in, yA_in, zA_in = float(mol_xyz[1]), float(mol_xyz[2]), float(mol_xyz[3])
    xB_in, yB_in, zB_in = float(mol_xyz[9]), float(mol_xyz[10]), float(mol_xyz[11])
    xC_in, yC_in, zC_in = float(mol_xyz[13]), float(mol_xyz[14]), float(mol_xyz[15])

    # place data and coordinates into a dictionary then append to list
    data_dict = {"atomA" : atomA, "atomB": atomB, "atomC" : atomC, 
                 "Cx_in" : Cx_in, "Cy_in" : Cy_in, "Cz_in" : Cz_in,
                 "xA_in" : xA_in, "yA_in" : yA_in, "zA_in" : zA_in,
                 "xB_in" : xB_in, "yB_in" : yB_in, "zB_in" : zB_in,
                 "xC_in" : xC_in, "yC_in" : yC_in, "zC_in" : zC_in,}
    parsed_row = pd.Series(data = data_dict)
    parsed_rows.append(parsed_row)

# convert parsed rows to dataframe
parsed_data = pd.DataFrame(parsed_rows)
print(parsed_data)

[Cl][C]([Cl])[Cl]
[Cl][C]([Cl])[F]
[Cl][C]([Cl])[H]
[Cl][C]([F])[F]
[Cl][C]([F])[H]
[Cl][C]([H])[H]
[F][C]([F])[F]
[F][C]([F])[H]
[F][C]([H])[H]
[H][C]([H])[H]
  atomA atomB atomC     Cx_in     Cy_in     Cz_in     xA_in     yA_in  \
0    Cl    Cl    Cl -0.001093  0.011846  0.446038 -0.988809 -1.349515   
1    Cl    Cl     F  0.003736  0.129010  0.374778 -1.451115 -0.775843   
2    Cl    Cl     H  0.005265  0.218641  0.368101 -1.426947 -0.721455   
3    Cl     F     F -0.137634  0.007709  0.358084  1.589999 -0.014417   
4    Cl     F     H -0.150692  0.111810  0.343827  1.536966 -0.236099   
5    Cl     H     H -0.202896 -0.008396  0.285350  1.542752 -0.048692   
6     F     F     F  0.021437 -0.007033  0.331909 -0.626317  1.138558   
7     F     F     H  0.001236  0.082656  0.292379  1.142276 -0.599058   
8     F     H     H -0.079681  0.001598  0.296070  1.234436 -0.133695   
9     H     H     H -0.020822  0.007404  0.291813  0.974814 -0.307380   

      zA_in     xB_in     yB_in     

Now that the input XYZ coordinates are generated they need to be stored into the original excel dataframe for easy visual inspection. First, the relative path to the excel dataframe is defined and imported as df. A new dataframe called merged is created using the merge method to merge df and parsed_data. The merge method uses the keys from the original dataframe, df, then merges only when atomA, atomB, and atomC indexes match between df and parsed_data. The new coordinates are added to coordinate columns with the _DELETE suffix. For example the xB_in coordinate is added to the column xB_in_DELETE while the xB_in column is left blank. Then the merged dataframe is updated with the parsed data which fills in the coordinates to the appropriate column. Finally all the columns with _DELETE suffix are deleted from the merged dataframe resulting in the original dataframe but with the correct coordinates corresponding to the correct atoms. Merging this way is computationally expensive but safer than using pandas update method alone which would not work if the parsed_data dataframe is out of order compared to the original dataframe.

In [341]:
#############################################################
# WRITE DATA TO DATA FRAME AND EXCEL
#############################################################


# reload original input dataframe (or you could make a completely new one)
CX3_dataframe = ".\\dataframes\\CX3_radical_dataframe.xlsx"
CX3_dataframe_test = ".\\dataframes\\CX3_radical_dataframe_test.xlsx"
# read file into pandas DataFrame object
df = pd.read_excel(CX3_dataframe)
df = df.sort_values(["atomA","atomB","atomC"], ignore_index= True)

# merge the parsed dataframe with the input dataframe
# merged = pd.merge(df, parsed_data, how="left", on=["atomA", "atomB", "atomC"], suffixes=["", "_DELETE"])
df.update(parsed_data)
# drop the duplicate columns
# merged = merged.loc[:, ~merged.columns.str.contains('DELETE')]
print(df)
# update the dataframe excel file with new data
# note this will over-write the current file, so any formatting or plots will be lost
# you could save to a new name to avoid this issue
df.to_excel(CX3_dataframe_test, index=False)

  atomA atomB atomC  nH  nF  nCl     Cx_in     Cy_in     Cz_in     xA_in  ...  \
0    Cl    Cl    Cl   0   0    3 -0.001093  0.011846  0.446038 -0.988809  ...   
1    Cl    Cl     F   0   1    2  0.003736  0.129010  0.374778 -1.451115  ...   
2    Cl    Cl     H   1   0    2  0.005265  0.218641  0.368101 -1.426947  ...   
3    Cl     F     F   0   2    1 -0.137634  0.007709  0.358084  1.589999  ...   
4    Cl     F     H   1   1    1 -0.150692  0.111810  0.343827  1.536966  ...   
5    Cl     H     H   2   0    1 -0.202896 -0.008396  0.285350  1.542752  ...   
6     F     F     F   0   3    0  0.021437 -0.007033  0.331909 -0.626317  ...   
7     F     F     H   1   2    0  0.001236  0.082656  0.292379  1.142276  ...   
8     F     H     H   2   1    0 -0.079681  0.001598  0.296070  1.234436  ...   
9     H     H     H   3   0    0 -0.020822  0.007404  0.291813  0.974814  ...   

   sdI  eprC                         eprH  eprF  eprCl  eprBr  eprI  \
0  NaN   NaN                         

The next step is to load the template file and pull the template keys used as variables in the dataframe. The template keys correspond to columns in the dataframe that this notebook pulls and imports data. The template keys are printed for the user's convenience.

In [342]:
#############################################################
# LOAD TEMPLATE
#############################################################

# define relative path to template file
CX3_radical_template = ".\\templates\\CX3_radical_template.txt"
# read template as a string
with open(CX3_radical_template, "r") as f:
    CX3_radical_template = f.read()
# get the set of required template keys
regex = re.compile(r'(?<!\{)\{([^{}]+)\}(?!\})') # Looks through template file for all strings inside {}
matches = set(regex.findall(CX3_radical_template))
template_keys = {item.split(":")[0] for item in matches}

print(template_keys)

{'Cz_in', 'molecule', 'zB_in', 'atomA', 'yA_in', 'zC_in', 'atomB', 'atomC', 'Cy_in', 'zA_in', 'yB_in', 'xC_in', 'yC_in', 'Cx_in', 'xA_in', 'xB_in'}


Before creating input files, the input data is given a file name and is filtered for any molecules missing information. The excel dataframe is loaded then a new column containing the molecule's file name is added. The dataframe is checked to ensure that all values associated with the template keys are not nan. Checking for nan ensures that all the input files are valid and will not through errors when run using Orca, Gaussian, ADF, ect simply because values are missing. A mask of all the non nan values is stored as complete_mask and the dataframe is redefined to only include such values.

In [343]:
#############################################################
# LOAD INPUT DATA
#############################################################

# define relative path to input dataframe
CX3_dataframe_input = ".\\dataframes\\CX3_radical_dataframe_test.xlsx"
# read file into pandas DataFrame object
df = pd.read_excel(CX3_dataframe_input)
# add molecule name column (format or change name however you like)
df["molecule"] = ["C-{atomA}-{atomB}-{atomC}_radical".format(**row) for _,row in df.iterrows()]

# check that each key required by the template has values that are not nan
complete_column_check = [df[key].notna() for key in template_keys]
# create a mask of complete rows to drop incomplete rows
complete_mask = (sum(complete_column_check) / len(template_keys)) >= 1
# only keep rows that have non-nan values for each required element of the template
df = df[complete_mask]

Next is to generate the input files using the previously verified data. First, directory is made to store the input files. Then the program interates through each molecule in the dataframe. For each molecule a file is written named with the molecule key and formatted using the CX3_radical_template. For a visual check, the program prints where it is writing each file.

In [312]:
#############################################################
# WRITE INPUT FILES FROM TEMPLATE AND DATA
#############################################################

# make a directory to store the input files
os.makedirs(".\\input_files\\CX3_radical", exist_ok=True)

# loop through dataframe and write an input file for each line
for _, row in df.iterrows():

    # use molecule name to define filename with relative path
    filename = ".\\input_files\\CX3_radical\\{molecule}.inp".format(**row)

    print(f"Now writing {filename}")
    # generate input files
    with open(filename, "w") as f:
        f.writelines(CX3_radical_template.format(**row))

Now writing .\input_files\CX3_radical\C-H-H-H_radical.inp
Now writing .\input_files\CX3_radical\C-F-H-H_radical.inp
Now writing .\input_files\CX3_radical\C-Cl-H-H_radical.inp
Now writing .\input_files\CX3_radical\C-F-F-H_radical.inp
Now writing .\input_files\CX3_radical\C-Cl-F-H_radical.inp
Now writing .\input_files\CX3_radical\C-Cl-Cl-H_radical.inp
Now writing .\input_files\CX3_radical\C-F-F-F_radical.inp
Now writing .\input_files\CX3_radical\C-Cl-F-F_radical.inp
Now writing .\input_files\CX3_radical\C-Cl-Cl-F_radical.inp
Now writing .\input_files\CX3_radical\C-Cl-Cl-Cl_radical.inp


At this point the user would take the input files and run them using Orca, Gaussian, ADF, ect. For this example, ORCA was used to run a Geometry Optimization on the input geometries. The template is set such that ORCA outputs an xyzfile of the optimiazed geometry.

The next section of the code is dependant on ORCA's output notation but can be tweaked for other programs. The output path is specified with out_path and the parsed_rows array is defined to store output geometries of each molecule similar to how the input geometries were stored. This code block loops through all the .out files in the out_path directory and uses the cclib library to read the atomic coordinates for each molecule. The atom_symbols dictionary is used to correctly index each atom to its atomic number for cclib. The atom identities and coordinates once stored as variables are then stored in the data_dict dictionary. The dictionary containing the molecules atoms and coordinates is converted into a pandas series which is appended to the parsed_rows array. Once the for loop iterates through all the output files, parsed_rows is converted into a pandas dataframe called parsed_data.

In [345]:
#############################################################
# LOAD OUTPUT FILES AND SCRAPE DATA
#############################################################

# define relative path to output files
out_path = ".\\output_files\\CX3_radical"
# loop through directory and load appropriate output files
parsed_rows = []
for folder in os.listdir(out_path):
    folder_path = "\\".join([out_path, folder])

    # use the cclib library to read the output file
    out_file = "\\".join([folder_path, folder + ".out"])
    ccout = cc.ccread(out_file)
    # check the documentation to see what other data you might want that these libraries will automatically supply
    # you can also choose to write your own parser, or find the data manually (especially for the property.txt files)


    # grab the data you want
    # you may choose to compute bond angles and other values here, rather than in excel
    atom_symbols = {"6" : "C", "1" : "H", "17" : "Cl", "9" : "F"}
    _, atomA, atomB, atomC = [atom_symbols[str(num)] for num in ccout.atomnos]
    Cx_out, Cy_out, Cz_out = ccout.atomcoords[-1][0]
    xA_out, yA_out, zA_out = ccout.atomcoords[-1][1]
    xB_out, yB_out, zB_out = ccout.atomcoords[-1][2]
    xC_out, yC_out, zC_out = ccout.atomcoords[-1][3]

    # append parsed row to list
    data_dict = {"atomA" : atomA, "atomB": atomB, "atomC" : atomC, 
                 "Cx_out" : Cx_out, "Cy_out" : Cy_out, "Cz_out" : Cz_out,
                 "xA_out" : xA_out, "yA_out" : yA_out, "zA_out" : zA_out,
                 "xB_out" : xB_out, "yB_out" : yB_out, "zB_out" : zB_out,
                 "xC_out" : xC_out, "yC_out" : yC_out, "zC_out" : zC_out,}
    parsed_row = pd.Series(data = data_dict)
    parsed_rows.append(parsed_row)

# convert parsed rows to dataframe
parsed_rows.reverse()
parsed_data = pd.DataFrame(parsed_rows)
print(parsed_data)

  atomA atomB atomC    Cx_out    Cy_out    Cz_out    xA_out    yA_out  \
0     H     H     H  0.000187 -0.000296  0.000067  1.051830 -0.242963   
1     F     H     H -0.092219  0.000880  0.109762  1.231268 -0.002992   
2     F     F     H -0.001412  0.078346  0.242303  1.099869 -0.578467   
3     F     F     F -0.000524 -0.001070  0.295616 -0.731346 -1.016090   
4    Cl     H     H -0.168717  0.001518 -0.000770  1.516637 -0.013888   
5    Cl     F     H -0.116008  0.089733  0.206337  1.531236 -0.234733   
6    Cl     F     F -0.103297  0.003684  0.288204  1.576829 -0.074866   
7    Cl    Cl     H -0.000416  0.152737  0.131320  1.463555 -0.675848   
8    Cl    Cl     F -0.002313  0.089421  0.264311  1.481661 -0.682382   

     zA_out    xB_out    yB_out    zB_out    xC_out    yC_out    zC_out  
0 -0.000889 -0.736138 -0.789446 -0.000193 -0.315880  1.031705  0.001015  
1 -0.033059 -0.571695 -0.957187 -0.031985 -0.567355  0.959299 -0.043718  
2 -0.074851 -1.080333 -0.614307 -0.074653 -0.01

In [346]:
print(parsed_data.loc[:,["atomA","atomB","atomC"]])
print(parsed_data.columns.get_indexer(["atomA","atomB","atomC"]))
idx = pd.Index(parsed_data.loc[:,["atomA","atomB","atomC"]])
print(idx)

  atomA atomB atomC
0     H     H     H
1     F     H     H
2     F     F     H
3     F     F     F
4    Cl     H     H
5    Cl     F     H
6    Cl     F     F
7    Cl    Cl     H
8    Cl    Cl     F
[0 1 2]
Index([  ('H', 'H', 'H'),   ('F', 'H', 'H'),   ('F', 'F', 'H'),
         ('F', 'F', 'F'),  ('Cl', 'H', 'H'),  ('Cl', 'F', 'H'),
        ('Cl', 'F', 'F'), ('Cl', 'Cl', 'H'), ('Cl', 'Cl', 'F')],
      dtype='object')


In [348]:
#############################################################
# WRITE DATA TO DATA FRAME AND EXCEL
#############################################################

# reload original input dataframe (or you could make a completely new one)
CX3_dataframe_test = ".\\dataframes\\CX3_radical_dataframe_test.xlsx"
CX3_dataframe_test2 = ".\\dataframes\\CX3_radical_dataframe_test2.xlsx"
# read file into pandas DataFrame object
df = pd.read_excel(CX3_dataframe_test)
df = df.sort_values(["atomA","atomB","atomC"], ignore_index= True)

# merge the parsed dataframe with the input dataframe
print(df)
df.update(parsed_data)
print(df)
# drop the duplicate columns

# update the dataframe excel file with new data
# note this will over-write the current file, so any formatting or plots will be lost
# you could save to a new name to avoid this issue
df.to_excel(CX3_dataframe_test2, index=False)

  atomA atomB atomC  nH  nF  nCl     Cx_in     Cy_in     Cz_in     xA_in  ...  \
0    Cl    Cl    Cl   0   0    3 -0.001093  0.011846  0.446038 -0.988809  ...   
1    Cl    Cl     F   0   1    2  0.003736  0.129010  0.374778 -1.451115  ...   
2    Cl    Cl     H   1   0    2  0.005265  0.218641  0.368101 -1.426947  ...   
3    Cl     F     F   0   2    1 -0.137634  0.007709  0.358084  1.589999  ...   
4    Cl     F     H   1   1    1 -0.150692  0.111810  0.343827  1.536966  ...   
5    Cl     H     H   2   0    1 -0.202896 -0.008396  0.285350  1.542752  ...   
6     F     F     F   0   3    0  0.021437 -0.007033  0.331909 -0.626317  ...   
7     F     F     H   1   2    0  0.001236  0.082656  0.292379  1.142276  ...   
8     F     H     H   2   1    0 -0.079681  0.001598  0.296070  1.234436  ...   
9     H     H     H   3   0    0 -0.020822  0.007404  0.291813  0.974814  ...   

   sdI  eprC                         eprH  eprF  eprCl  eprBr  eprI  \
0  NaN   NaN                         